In [1]:
# coding: utf-8
from heapq import nlargest
from operator import itemgetter
from collections import defaultdict

In [2]:
print('Preparing arrays...')
filename = 'train'
f = open( r"..\expedia_data\{0}.csv".format(filename) )
f.readline()

Preparing arrays...


'date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster\n'

In [3]:
best_hotels_od_ulc = defaultdict(lambda: defaultdict(int))
best_hotels_search_dest = defaultdict(lambda: defaultdict(int))
best_hotels_search_dest1 = defaultdict(lambda: defaultdict(int))
best_hotel_country = defaultdict(lambda: defaultdict(int))
popular_hotel_cluster = defaultdict(int)
total = 0

In [4]:
# Calc counts
while 1:
    line = f.readline().strip()
    total += 1

    if total % 100000 == 0:
        print('Read {} lines...'.format(total))

    if line == '':
        break

    arr = line.split(",")
    book_year = int(arr[0][:4])
    user_location_city = arr[5]
    orig_destination_distance = arr[6]
    srch_destination_id = arr[16]
    is_booking = int(arr[18])
    hotel_country = arr[21]
    hotel_market = arr[22]
    hotel_cluster = arr[23]

    append_1 = 3 + 12*is_booking
    append_2 = 3 + 5*is_booking

    if user_location_city != '' and orig_destination_distance != '':
        hsh = (hash('user_location_city_'+str(user_location_city) +
                    '_orig_destination_distance_' +
                    str(orig_destination_distance)))
        best_hotels_od_ulc[hsh][hotel_cluster] += 1

    if srch_destination_id != '' and hotel_country != '' \
            and hotel_market != '' and book_year == 2014:
        hsh = (hash('srch_destination_id_' +
                    str(srch_destination_id) +
                    "_hotel_country_" + str(hotel_country) +
                    "_hotel_market_"+str(hotel_market)))
        best_hotels_search_dest[hsh][hotel_cluster] += append_1
    if srch_destination_id != '':
        hsh = hash('srch_destination_id_'+str(srch_destination_id))
        best_hotels_search_dest1[hsh][hotel_cluster] += append_1
    if hotel_country != '':
        hsh = hash('hotel_country_'+str(hotel_country))
        best_hotel_country[hsh][hotel_cluster] += append_2
    popular_hotel_cluster[hotel_cluster] += 1
f.close()

Read 100000 lines...
Read 200000 lines...
Read 300000 lines...
Read 400000 lines...
Read 500000 lines...
Read 600000 lines...
Read 700000 lines...
Read 800000 lines...
Read 900000 lines...
Read 1000000 lines...
Read 1100000 lines...
Read 1200000 lines...
Read 1300000 lines...
Read 1400000 lines...
Read 1500000 lines...
Read 1600000 lines...
Read 1700000 lines...
Read 1800000 lines...
Read 1900000 lines...
Read 2000000 lines...
Read 2100000 lines...
Read 2200000 lines...
Read 2300000 lines...
Read 2400000 lines...
Read 2500000 lines...
Read 2600000 lines...
Read 2700000 lines...
Read 2800000 lines...
Read 2900000 lines...
Read 3000000 lines...
Read 3100000 lines...
Read 3200000 lines...
Read 3300000 lines...
Read 3400000 lines...
Read 3500000 lines...
Read 3600000 lines...
Read 3700000 lines...
Read 3800000 lines...
Read 3900000 lines...
Read 4000000 lines...
Read 4100000 lines...
Read 4200000 lines...
Read 4300000 lines...
Read 4400000 lines...
Read 4500000 lines...
Read 4600000 lines.

In [5]:
print('Generate submission...')
filename = 'low_mem_submission'
out = open(r"..\expedia_submissions\{0}.csv".format(filename), "w")
filename = 'test'
f = open( r"..\expedia_data\{0}.csv".format(filename) )
f.readline()
total = 0
out.write("id,hotel_cluster\n")
topclusters = nlargest(5, sorted(popular_hotel_cluster.items()),
                       key=itemgetter(1))

Generate submission...


In [6]:
while 1:
    line = f.readline().strip()
    total += 1

    if total % 100000 == 0:
        print('Write {} lines...'.format(total))

    if line == '':
        break

    arr = line.split(",")
    id = arr[0]
    user_location_city = arr[6]
    orig_destination_distance = arr[7]
    srch_destination_id = arr[17]
    hotel_country = arr[20]
    hotel_market = arr[21]

    out.write(str(id) + ',')
    filled = []

    hsh = (hash('user_location_city_'+str(user_location_city) +
                '_orig_destination_distance_' +
                str(orig_destination_distance)))
    if hsh in best_hotels_od_ulc:
        d = best_hotels_od_ulc[hsh]
        topitems = nlargest(5, sorted(d.items()), key=itemgetter(1))
        for i in range(len(topitems)):
            if topitems[i][0] in filled:
                continue
            if len(filled) == 5:
                break
            out.write(' ' + topitems[i][0])
            filled.append(topitems[i][0])

    hsh1 = (hash('srch_destination_id_' +
                 str(srch_destination_id) +
                 "_hotel_country_" + str(hotel_country) +
                 "_hotel_market_"+str(hotel_market)))
    hsh2 = hash('srch_destination_id_'+str(srch_destination_id))
    if (len(filled) < 5) and (hsh1 in best_hotels_search_dest):
        d = best_hotels_search_dest[hsh1]
        topitems = nlargest(5, d.items(), key=itemgetter(1))
        for i in range(len(topitems)):
            if topitems[i][0] in filled:
                continue
            if len(filled) == 5:
                break
            out.write(' ' + topitems[i][0])
            filled.append(topitems[i][0])
    elif (len(filled) < 5) and (hsh2 in best_hotels_search_dest1):
        d = best_hotels_search_dest1[hsh2]
        topitems = nlargest(5, d.items(), key=itemgetter(1))
        for i in range(len(topitems)):
            if topitems[i][0] in filled:
                continue
            if len(filled) == 5:
                break
            out.write(' ' + topitems[i][0])
            filled.append(topitems[i][0])

    hsh = hash('hotel_country_'+str(hotel_country))
    if (len(filled) < 5) and (hsh in best_hotel_country):
        d = best_hotel_country[hsh]
        topitems = nlargest(5, d.items(), key=itemgetter(1))
        for i in range(len(topitems)):
            if topitems[i][0] in filled:
                continue
            if len(filled) == 5:
                break
            out.write(' ' + topitems[i][0])
            filled.append(topitems[i][0])

    if(len(filled) < 5):
        for i in range(len(topclusters)):
            if topclusters[i][0] in filled:
                continue
            if len(filled) == 5:
                break
            out.write(' ' + topclusters[i][0])
            filled.append(topclusters[i][0])

    out.write("\n")

Write 100000 lines...
Write 200000 lines...
Write 300000 lines...
Write 400000 lines...
Write 500000 lines...
Write 600000 lines...
Write 700000 lines...
Write 800000 lines...
Write 900000 lines...
Write 1000000 lines...
Write 1100000 lines...
Write 1200000 lines...
Write 1300000 lines...
Write 1400000 lines...
Write 1500000 lines...
Write 1600000 lines...
Write 1700000 lines...
Write 1800000 lines...
Write 1900000 lines...
Write 2000000 lines...
Write 2100000 lines...
Write 2200000 lines...
Write 2300000 lines...
Write 2400000 lines...
Write 2500000 lines...


In [7]:
out.close()
print('Completed!')

Completed!
